<a href="https://colab.research.google.com/github/ambika1us/Hate-Speech-Detection-using-LSTM/blob/main/Hate_Speech_Detection_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#Import Importent Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import spacy
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
df=pd.read_csv('/content/drive/MyDrive/Hate Speech Detection using LSTM/labeled_data.csv')

In [37]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [38]:
df.isna().sum()

,0
Unnamed: 0,0
count,0
hate_speech,0
offensive_language,0
neither,0
class,0
tweet,0


In [39]:
df.columns

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')

In [40]:
df.drop(columns=['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'], inplace=True)

In [41]:
df

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [42]:
#Deleting unwanted symbols & Numeric data
df['Processed_tweet'] = df['tweet'].str.replace("[^a-zA-Z#]", " ",regex=True)

In [43]:
#Handling unwanted space
df['Processed_tweet_2']=df['Processed_tweet'].str.replace('\s+', ' ',regex=True)

In [44]:
df['Processed_tweet'].iloc[0]

'    RT  mayasolovely  As a woman you shouldn t complain about cleaning up your house   amp  as a man you should always take the trash out   '

In [45]:
df['Processed_tweet_2'].iloc[0]

' RT mayasolovely As a woman you shouldn t complain about cleaning up your house amp as a man you should always take the trash out '

In [46]:
#Deleting unwanted columns
df.drop(columns=['tweet','Processed_tweet'], inplace=True)

In [47]:
#NLP
nlp=spacy.load('en_core_web_sm')

In [48]:
#Lemmatization
def lemmatization(text):
  doc=nlp(text)
  lemmalist=(word.lemma_ for word in doc)
  return " ".join(lemmalist)

In [49]:
df['lemma_tweet']=df['Processed_tweet_2'].apply(lemmatization)

In [50]:
#Remove the stopwords
def remove_stopwords(text):
  doc=nlp(text)
  no_stopword_list=(word.text for word in doc if not word.is_stop)
  return " ".join(no_stopword_list)

In [51]:
df['Final_tweet']=df['lemma_tweet'].apply(remove_stopwords)

In [52]:
#One Hot represention
vocab_size=10000
onehot_repr=[one_hot(words,vocab_size) for words in df['Final_tweet']]

In [53]:
sentence_length=20
embedded_tweet=pad_sequences(onehot_repr,padding='pre',maxlen=sentence_length)

In [54]:
for i in range(0,4):
  print(embedded_tweet[i])

[   0    0    0    0    0    0    0    0    0 4450 6897 2094 7715 4924
 5986 6218 9246 2991 4323 2321]
[   0    0    0    0    0    0    0 4450 8131 7556  959 8792 7394 3422
 1751 6152  959 8518 2568 3922]
[   0    0    0    0    0    0    0    0 4450 8894 1433 4450  574 7626
 8059  310 2238 9473 9412 2467]
[   0    0    0    0    0    0    0    0    0    0    0 4450 5342 1847
 3935 5939 3803 5196 6021 6544]


In [55]:
X=np.array(embedded_tweet)
y=np.array(df['class'])

In [56]:
df['class'].value_counts()

,count
class,
1,19190
2,4163
0,1430


In [57]:
smote=SMOTE(sampling_strategy='minority')
X,y=smote.fit_resample(X,y)

In [58]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [59]:
#Creating Models
dimension=50
model=keras.Sequential([
    keras.layers.Embedding(vocab_size,dimension,input_length=sentence_length),
    keras.layers.LSTM(100,return_sequences=True),
    keras.layers.LSTM(50,return_sequences=True),
    keras.layers.LSTM(50),
    keras.layers.Dense(3,activation='softmax')
                                               ])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.fit(X_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 56s 46ms/step - accuracy: 0.7785 - loss: 0.5073
Epoch 2/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 81s 46ms/step - accuracy: 0.9455 - loss: 0.1721
Epoch 3/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 87s 51ms/step - accuracy: 0.9632 - loss: 0.1194
Epoch 4/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 49s 46ms/step - accuracy: 0.9727 - loss: 0.0927
Epoch 5/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 83s 47ms/step - accuracy: 0.9780 - loss: 0.0733
Epoch 6/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 82s 47ms/step - accuracy: 0.9838 - loss: 0.0509
Epoch 7/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 81s 46ms/step - accuracy: 0.9883 - loss: 0.0365
Epoch 8/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 49s 46ms/step - accuracy: 0.9925 - loss: 0.0234
Epoch 9/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.9938 - loss: 0.0201
Epoch 10/10
1064/1064 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.9960 - loss: 0.0143


In [63]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Loss: {loss*100}")
print(f"Accuracy: {accuracy*100}")

266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8886 - loss: 0.5829
Loss: 59.133440256118774
Accuracy: 88.8471007347107


In [64]:
pred=np.argmax(model.predict(X_test),axis=-1)

266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step


In [65]:
y_test[:5]

array([1, 1, 0, 1, 2])

In [66]:
pred[:5]

array([1, 1, 0, 1, 2])